In [80]:
import os
import music21 as m21
import json
import tensorflow.keras as keras
import numpy as np

KERN_DATASET_PATH = "./deutschl/test"
DATASET_PATH = "dateset"
SINGLE_FILE_PATH = "file_dataset"

SEQUENCE_LENGTH = 64
MAPPING_PATH = "mapping.json"

ACCEPTABLE_DURATIONS = [
	0.25,
	0.5,
	1.0,
	1.5,
	2, 
	3,
	4
]

2023-05-15 00:10:31.191723: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 00:10:31.382759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 00:10:31.384083: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 00:10:32.185188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [83]:

def load_songs_in_kern(dataset_path):
    songs = []
    for path, subdirs, files in os.walk(dataset_path):
        for file in files:
            if file[-3:] == "krn":
                song = m21.converter.parse(os.path.join(path, file))
                songs.append((song, file.split(".")[0]))
                
    return songs

def is_acceptable_song(song, acceptable_duration):
    # for note in song.flat.notesAndRests:
    #     if note.duration.quarterLength not in acceptable_duration:
    #         return False
        
    return True

def transpose(song):
    # get key from song
    parts = song.getElementsByClass(m21.stream.Part)
    mearsure_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = mearsure_part0[0][4]
    
    # using m21 to estimate key
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")
        
    # get interval for transposition
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))
        
    # reduce all key to C major or A minor to make learning more easy
    transposed_song = song.transpose(interval)
    
    return transposed_song
    
def encode_song(song, time_step = 0.25):
    # p = 60, d = 1.0 -> [60, _, _, _]
    
    encoded_song = []
    for event in song.flat.notesAndRests:
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi
        elif isinstance(event, m21.note.Rest):
            symbol = "r"
            
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")
                
    # cast encoded song to a str
    encoded_song = " ".join(map(str, encoded_song))            
    return encoded_song 
        

def preprocess(dataset_path):
    print(f"Loading songs...")
    songs = load_songs_in_kern(dataset_path)
    print(f"Loaded {len(songs)} songs")
    
    for i, song_info in enumerate(songs):
        song = song_info[0]
        if not is_acceptable_song(song, ACCEPTABLE_DURATIONS):
            print("not acceptable: ", song_info[1])
            continue
        
        song = transpose(song)
        
        encoded_song = encode_song(song)
        
        save_path = os.path.join(DATASET_PATH, song_info[1])
        with open(save_path, "w") as fp:
            fp.write(encoded_song)
            
def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song
        

def create_single_file_dataset(dataset_path, file_dataset_path):
    new_song_delimeter = "/ " * SEQUENCE_LENGTH
    songs = ""
    
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            song = load(file_path)
            
            songs = songs + song + " " + new_song_delimeter
            
    songs = songs[: -1]
    with open(file_dataset_path, "w") as fp:
        fp.write(songs)
        
    return songs

def create_mapping(song_str, mapping_path):
    mappings = {}
    
    songs = song_str.split()
    
    print(len(songs))
    vocabulary = list(set(songs))
    
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i
        
    with open(mapping_path, "w") as fp:
        json.dump(mappings, fp, indent= 4)
        
def convert_songs_to_int(encoded_songs):
    int_symbols = []
    
    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)
        
    symbols = encoded_songs.split()
    
    for symbol in symbols:
        int_symbols.append(mappings[symbol])
        
    return int_symbols

def generating_training_sequences(sequence_length):
    songs = load(SINGLE_FILE_PATH)
    int_songs = convert_songs_to_int(songs)
    
    inputs = []
    targets = []
    seq_num = len(int_songs) - sequence_length
    
    for i in range(seq_num):
        inputs.append(int_songs[i: i + sequence_length])
        targets.append(int_songs[i + sequence_length])
        
    # encoding with one-hot
    # inputs : (seq_num, sequence_length)
    # [[0, 1, 2], [1, 1, 2]] -> [[[1, 0, 0], [0, 1, 0], [0, 0, 1]], [[0, 1, 0], [0, 1, 0], [0, 0, 1]]]
     
    # targets: (seq_num, 1)
    vacubulary_size = len(set(int_songs))
    inputs = keras.utils.to_categorical(inputs, num_classes = vacubulary_size)
    targets = np.array(targets)
    
    return inputs, targets
    
    
    

In [ ]:
# for test
# songs = load_songs_in_kern(KERN_DATASET_PATH)
# print(f"Loaded {len(songs)} songs.")

# song_info = songs[0]
# song = song_info[0]

# print("show song: ", song_info[1])
# song.show()
# song.show('midi')

# transposed_song = transpose(song)
# transposed_song.show()
# transposed_song.show('midi')


In [87]:

# main procedure
preprocess(KERN_DATASET_PATH)
songs = create_single_file_dataset(DATASET_PATH, SINGLE_FILE_PATH)
create_mapping(songs, MAPPING_PATH)

inputs, targets = generating_training_sequences(SEQUENCE_LENGTH)

print(inputs.shape)
print(targets.shape)


Loading songs...
Loaded 12 songs
2576
(2512, 64, 18)
(2512,)
